In [1]:
from keras.layers.core import Flatten
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, RepeatVector
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.layers import Merge
from keras.optimizers import SGD
from keras.layers.core import Activation
import numpy as np
import h5py

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5105)


In [2]:
image_model = Sequential()
image_model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
image_model.add(Convolution2D(64, 3, 3, activation='relu'))
image_model.add(ZeroPadding2D((1,1)))
image_model.add(Convolution2D(64, 3, 3, activation='relu'))
image_model.add(MaxPooling2D((2,2), strides=(2,2)))

image_model.add(ZeroPadding2D((1,1)))
image_model.add(Convolution2D(128, 3, 3, activation='relu'))
image_model.add(ZeroPadding2D((1,1)))
image_model.add(Convolution2D(128, 3, 3, activation='relu'))
image_model.add(MaxPooling2D((2,2), strides=(2,2)))

image_model.add(ZeroPadding2D((1,1)))
image_model.add(Convolution2D(256, 3, 3, activation='relu'))
image_model.add(ZeroPadding2D((1,1)))
image_model.add(Convolution2D(256, 3, 3, activation='relu'))
image_model.add(ZeroPadding2D((1,1)))
image_model.add(Convolution2D(256, 3, 3, activation='relu'))
image_model.add(MaxPooling2D((2,2), strides=(2,2)))

image_model.add(ZeroPadding2D((1,1)))
image_model.add(Convolution2D(512, 3, 3, activation='relu'))
image_model.add(ZeroPadding2D((1,1)))
image_model.add(Convolution2D(512, 3, 3, activation='relu'))
image_model.add(ZeroPadding2D((1,1)))
image_model.add(Convolution2D(512, 3, 3, activation='relu'))
image_model.add(MaxPooling2D((2,2), strides=(2,2)))

image_model.add(ZeroPadding2D((1,1)))
image_model.add(Convolution2D(512, 3, 3, activation='relu'))
image_model.add(ZeroPadding2D((1,1)))
image_model.add(Convolution2D(512, 3, 3, activation='relu'))
image_model.add(ZeroPadding2D((1,1)))
image_model.add(Convolution2D(512, 3, 3, activation='relu'))
image_model.add(MaxPooling2D((2,2), strides=(2,2)))

image_model.add(Flatten())
image_model.add(Dense(4096, activation='relu'))
image_model.add(Dropout(0.5))
image_model.add(Dense(4096, activation='relu'))
# image_model.summary()

/usr/local/lib/python2.7/dist-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/usr/local/lib/python2.7/dist-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/usr/local/lib/python2.7/dist-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [3]:
f = h5py.File('vgg16_weights.h5')
for k in range(f.attrs['nb_layers']):
    if k >= 35:
        # we don't look at the last (fully-connected) layers in the savefile
        break
#     print k
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    image_model.layers[k].set_weights(weights)
f.close()
print "VGG Loaded with pre-trained weights."

VGG Loaded with pre-trained weights.


### Extracting Image Features

In [4]:
import scipy.io
Images = scipy.io.loadmat('Images.mat')['out']

In [5]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
image_model.compile(optimizer=sgd, loss='categorical_crossentropy')

In [6]:
Images = Images[:16200]
print type(Images), Images.shape

<type 'numpy.ndarray'> (1, 16200)


In [7]:
import time
# from tqdm import tqdm
feat = []
for i in range(0,810):
    feat = np.append(feat,image_model.predict(Images[i*20:(i*20+20)])) 
    time.sleep(1)

In [9]:
print "Saving Feature array"
matfile = 'Features.mat'
scipy.io.savemat(matfile, mdict={'out': feat}, oned_as='row')
print "Data Saved"

Saving Feature array
Data Saved


### Extracting Bounding box features

In [4]:
import scipy.io
bb_data = scipy.io.loadmat('baseline_bb.mat')['out']

In [5]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
image_model.compile(optimizer=sgd, loss='categorical_crossentropy')

In [6]:
bb_data = bb_data[:16200]
print bb_data.shape

(16200, 3, 224, 224)


In [7]:
import time
# from tqdm import tqdm
bb_feat = []
for i in range(0,162):
    bb_feat = np.append(bb_feat,image_model.predict(bb_data[i*100:(i*100+100)])) 
    time.sleep(1)

In [8]:
print "Savind bounding box features"
import scipy.io
matfile = 'baseline_bb_vgg.mat'
scipy.io.savemat(matfile, mdict={'out': bb_feat}, oned_as='row')
print "Data Saved"

Savind bounding box features
Data Saved
